In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

## Indicators

In [2]:
def VWAP(df, window=10):
    price = df[['High', 'Low', 'Close']].mean(1)
    vol = df['Volume']
    return (price * vol).rolling(window).sum() / vol.rolling(window).sum()

In [3]:
def ATR(df, window=14):
    tr = df[['High', 'Low', 'Close']].apply(
        lambda x: x.max() - x.min(), axis=1)
    return tr.rolling(window).mean()

In [4]:
def BollingerBands(df, window=20, m=2):
    sma = df['Close'].rolling(window).mean()
    std = df['Close'].rolling(window).std()
    df['Upper_BB'] = sma + (std * m)
    df['Lower_BB'] = sma - (std * m)
    df['Middle_BB'] = sma
    return df

In [5]:
def get_date(date, s, e, include_end=False):
    if include_end:
        return date[(date >= s) & (date <= e)]
    return date[(date >= s) & (date < e)]

In [6]:
def groupby(df, freq):
    return df.groupby(by=pd.Grouper(freq=freq))

def regroup(df, freq='W'):
    o = groupby(df['Open'], freq).first()
    h = groupby(df['High'], freq).max()
    l = groupby(df['Low'], freq).min()
    c = groupby(df['Close'], freq).last()
    grouped = pd.concat([o, h, l, c], axis=1)
    grouped.columns = ['Open', 'High', 'Low', 'Close']
    return grouped

In [7]:
def status(df):
    ys = df.groupby(by=pd.Grouper(freq='Y'))
    concat = pd.concat([ys.mean(),
                        ys.prod(),
                        ys.apply(lambda x: np.mean(x > 1))], axis=1)
    concat.columns = ['avg_gain', 'return', 'win_rate']
    concat.index = [idx.year for idx in concat.index]
    concat.loc['D'] = [concat['avg_gain'].mean(), 
                       concat['return'].prod(), 
                       concat['win_rate'].mean()]
    return concat

In [71]:
def cal_pct(df, tp: float):
    pct_max = (df['High'] / df['Close'].shift()).dropna()
    pct =  (df['Close'] / df['Close'].shift()).dropna()
    
    m = pct_max > (1 + tp)
    pct[m] = pct_max[m] * (1 - tp)
    return pct

In [72]:
def get_status(df, s, e, tp):
    data = cal_pct(df, tp=tp)
    date = get_date(data.index, s, e, True)
    return status(data[date])

## Daily figures

In [10]:
import os

cwd = os.getcwd()
# Check if price data have been saved
price_data = os.path.join(cwd, "price_data")
if not os.path.exists(price_data):
    # Getting the Nasdaq 100 ticker symbols from Wikipedia
    nas_df = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
    tickers = nas_df.Ticker.to_list()
    # Download the Nasdaq 100 Adjusted Close price data with yfinance
    df = yf.download(tickers, start="2010-01-01")
    # Create price_data directory
    os.makedirs(price_data)
    # Save a copy of the dataframe to price_data directory
    df.to_csv(os.path.join(price_data, "Nasdaq-100.csv"))
else:
    # Load the Nasdaq-100.csv
    df = pd.read_csv(os.path.join(price_data, "Nasdaq-100.csv"), 
                     header=[0,1], index_col=0)
    df.index = pd.to_datetime(df.index)

In [73]:
sdf = df.swaplevel(axis=1)

## Analysis

In [86]:
data = regroup(sdf['TSLA'].copy(), 'W')

In [88]:
rang = np.arange(.01, 1, step=.01)
score = pd.DataFrame(columns=['avg_gain', 'return', 'win_rate'])
for i, r in enumerate(rang):
    stat = get_status(data, '2010-01-01', '2020-12-31', tp=r)
    score.loc[r] = [stat.loc['D', 'avg_gain'],
                    stat.loc['D', 'return'], 
                    stat.loc['D', 'win_rate']]
pct = score['return'].nlargest(1).index.values[0]

print(pct)
stat = get_status(data, '2021-01-01', '2022-12-31', tp=pct)
stat

0.01


,avg_gain,return,win_rate
2021,1.037948,6.031510,0.826923
2022,1.030355,4.049585,0.788462
D,1.034151,24.425116,0.807692
